In [23]:
!pip install datasets==1.18.3
!pip install transformers==4.17.0
!pip install eng-to-ipa
!pip install jiwer 

import eng_to_ipa as p
from datasets import load_dataset, load_metric
accuracy_metric = load_metric("accuracy")
wer_metric = load_metric("wer")
from google.colab import drive
drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#timit = load_dataset("timit_asr")
slue = load_dataset("asapp/slue", "voxceleb")


Downloading:   0%|          | 0.00/12.8k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.7k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.55G [00:00<?, ?B/s]

/root/.cache/huggingface/datasets/downloads/extracted/50bec784754ee4fb0f60e54e86b1f664529fdacf86e46524cf962ff59bf36f75/slue-voxceleb


0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset slue downloaded and prepared to /root/.cache/huggingface/datasets/asapp___slue/voxceleb/2.4.0/6eef0a9b9ce2693a4f0ffdf4dd74ab5dc6e95bcd07366ac2a2b275a5b296f90d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

phonem embeding normalized text

In [4]:
from transformers import Wav2Vec2CTCTokenizer

tokenizer = Wav2Vec2CTCTokenizer("/content/drive/MyDrive/vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [5]:
from transformers import Wav2Vec2FeatureExtractor

feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, 
                                             padding_value=0.0, do_normalize=True, 
                                             return_attention_mask=False)

In [6]:
from transformers import Wav2Vec2Processor

processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [7]:
def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_values"] = processor(audio["array"], sampling_rate=audio["sampling_rate"]).input_values[0]
    batch["input_length"] = len(batch["input_values"])
    
    with processor.as_target_processor():
        batch["labels_phonems"] = processor(p.convert(batch["normalized_text"])).input_ids
        batch['sentimen'] = batch['sentiment']

    return batch

In [8]:
dataset_train = slue.map(prepare_dataset, remove_columns=slue.column_names["train"],
                               num_proc=4)

In [9]:
arr = []
for i in range(len(slue['train']['sentiment'])):
    if slue['train']['sentiment'][i] == 'Negative':
       arr.append(0)
    elif slue['train']['sentiment'][i] == 'Neutral':
        arr.append(1)
    elif slue['train']['sentiment'][i] == 'Positive':
        arr.append(2)

for i in range(48):
    arr.append(3)
len(arr)
slue_train = dataset_train['train'].add_column(name="labels_sentiment", column=arr)
arr1 = []
for i in range(len(slue['validation']['sentiment'])):
    if slue['validation']['sentiment'][i] == 'Negative':
       arr1.append(0)
    elif slue['validation']['sentiment'][i] == 'Neutral':
        arr1.append(1)
    elif slue['validation']['sentiment'][i] == 'Positive':
        arr1.append(2)

for i in range(17):
    arr1.append(3)
len(arr1)
slue_test = dataset_train['validation'].add_column(name="labels_sentiment", column=arr1)

In [10]:
print(slue_train)
print(slue_test)


Dataset({
    features: ['input_values', 'input_length', 'labels_phonems', 'sentimen', 'labels_sentiment'],
    num_rows: 5777
})
Dataset({
    features: ['input_values', 'input_length', 'labels_phonems', 'sentimen', 'labels_sentiment'],
    num_rows: 1454
})


In [ ]:
#уменьшим число записей больше 5 секунд
max_input_length_in_sec = 5.0
slue_train = slue_train.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, 
                               input_columns=["input_length"])
slue_test = slue_test.filter(lambda x: x < max_input_length_in_sec * processor.feature_extractor.sampling_rate, 
                               input_columns=["input_length"])

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [11]:
slue_train = slue_train.remove_columns(['input_length'])
slue_test = slue_test.remove_columns(['input_length'])

In [12]:
slue_train = slue_train.remove_columns(['sentimen'])
slue_test = slue_test.remove_columns(['sentimen'])

In [3]:
label2id = {
    'Negative': 0,
 'Neutral': 1,
 'Positive': 2,
 'unknown' : 3

}
id2label = {
    0: 'Negative',
 1: 'Neutral',
 2: 'Positive',
 3: 'unknown'
}

In [ ]:
slue_train

Dataset({
    features: ['input_values', 'labels_phonems', 'labels_sentiment'],
    num_rows: 5777
})

In [13]:
slue_train.set_format('torch')
slue_test.set_format('torch')

In [14]:
import torch
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    -Сопоставитель данных, который будет динамически дополнять полученные входные данные.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
            -Процессор, используемый для обработки данных.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            -
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided). - Дополнение к самой длинной последовательности в пакете (или без заполнения, если только одна
              последовательность, если она указана).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided. - Дополнить до максимальной длины, указанной аргументом :obj:`max_length`, или до
              максимальная приемлемая длина входных данных для модели, если этот аргумент не указан.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels_phonems"]} for feature in features]
        label_features_ = [{"input_ids": feature["labels_sentiment"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                return_tensors="pt",
            )
            labels_sent_batch = self.processor.pad(
                label_features_,
                padding=self.padding,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels_phonems"] = labels
        batch["labels_sentiment"] = labels_sent_batch

        return batch

In [15]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [16]:
#создаем тестовую и обучающую выборку для последующего обучения
from torch.utils.data import DataLoader

train_DataLoader = DataLoader(slue_train, shuffle = True, batch_size = 8, collate_fn= data_collator)
test_DataLoader = DataLoader(slue_test, shuffle = True, batch_size = 8, collate_fn= data_collator)

In [ ]:
# /content/drive/MyDrive/multitask, facebook/wav2vec2-base-960h  -- названия моделей для обучения, первая уже немного натренирована

In [17]:
#класс нашей мультитаск модели
import torch
from torch import nn
from transformers import Wav2Vec2ForCTC
class Model_wav2vec(torch.nn.Module):
    def __init__(self, huggingface_model_name, head_in_features):
        super().__init__()
        self.wav2vec2 = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h",  label2id=label2id,
        id2label=id2label,
        output_hidden_states = True)
        #self.wav2vec2.config.feat_extract_activation = "softplus"
        #self.wav2vec2.feature_extractor.conv_layers.activation = nn.ReLU()
        self.wav2vec2.lm_head = torch.nn.Identity(head_in_features)

        self.phonemes_head = torch.nn.Linear(in_features= 768, out_features= 46, bias = True)

        self.sentiment_head = torch.nn.Linear(in_features= 768, out_features= len(label2id), bias = True)
    def save_pretrained(self,x):
        self.wav2vec2.save_pretrained(x)
    def forward(self, x):
        wav2vec2_out = self.wav2vec2(x)
        wav2vec2_hidden_states = wav2vec2_out.hidden_states
        phonemes_out = self.phonemes_head(wav2vec2_hidden_states[12])
        sentiment_out = self.sentiment_head(wav2vec2_hidden_states[6].mean(axis = 1))#менял на wav2vec2_hidden_states[10], потом точность при обучении возрастала
                                                                                    # примерно на 20 процентов и дальше не улучшалось

        return phonemes_out, sentiment_out

In [18]:
#создаем модель
mo = Model_wav2vec("", 768)
mo.to('cuda')

Downloading:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/360M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model_wav2vec(
  (wav2vec2): Wav2Vec2ForCTC(
    (wav2vec2): Wav2Vec2Model(
      (feature_extractor): Wav2Vec2FeatureEncoder(
        (conv_layers): ModuleList(
          (0): Wav2Vec2GroupNormConvLayer(
            (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
            (activation): GELUActivation()
            (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
          )
          (1): Wav2Vec2NoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
          (2): Wav2Vec2NoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
          (3): Wav2Vec2NoLayerNormConvLayer(
            (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
            (activation): GELUActivation()
          )
          (4): Wav2Vec2NoLayerNormConvLayer(
      

In [19]:
from transformers import AdamW

optimizer = AdamW(mo.parameters(), lr=1e-5)


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [20]:
from transformers import get_scheduler

num_epochs = 5
num_training_steps = num_epochs * len(train_DataLoader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

In [21]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
for data in train_DataLoader:
    data = data.to(device)
print(device)

cuda


In [22]:
#цикл обучения модели
from torch import logit
from tqdm.auto import tqdm
loss_ce = nn.CrossEntropyLoss(size_average = False,reduction = 'sum')
ctc_loss = nn.CTCLoss(zero_infinity=False, reduction = 'sum')
progress_bar = tqdm(range(num_training_steps))
#progress_train_bar = tqdm(range(len(test_DataLoader)))
loss_array = []
count = 0

mo.train()
for epoch in range(num_epochs):
    for batch in train_DataLoader:
        batch = {k: v.to(device) for k, v in batch.items()}
        mo.train()
        target_phonemes = batch['labels_phonems']
        target_sentimen = batch[ 'labels_sentiment']['input_ids']
        embedding = batch['input_values']
        batch_size =embedding.shape[0]
        
        mo.train()
        outputs_phonemes, output_sentiment = mo(embedding)
        loss1 = loss_ce(output_sentiment, target_sentimen)
        #loss1.backward()
        #loss_array.append(loss1)
        input = torch.nn.functional.log_softmax(outputs_phonemes.transpose(0,1),dim = 2)
        input_length = torch.full(size = (batch_size,), fill_value = input.shape[0], dtype = torch.long)
        target_phonem_length = (target_phonemes != 0).sum(axis = 1)
        loss2 = ctc_loss(input, target_phonemes, input_length, target_phonem_length)
        #loss2.backward()
        loss = loss1+loss2
        print(f"loss1: {loss1.item():.2f}; loss2: {loss2.item():.2f}; loss_sum: {loss.item():.5f}; iter: {count}")
        #print(f"loss: {loss.item():.5f}")
        count+=1
        
        loss.backward()
        optimizer.zero_grad()
        optimizer.step()
        lr_scheduler.step()
        torch.nn.utils.clip_grad_norm(parameters=mo.parameters(), max_norm=10, norm_type=2.0)
        progress_bar.update(1)

    mo.eval()
    for batch in test_DataLoader:
      batch = {k: v.to(device) for k, v in batch.items()}
      target_phonemes = batch['labels_phonems']
      target_sentimen = batch[ 'labels_sentiment']['input_ids']
      embedding = batch['input_values']
      with torch.no_grad():
        outputs_phonemes_, output_sentiment_  = mo(embedding)
        logits_phonem_ = logit(outputs_phonemes_)
        logits_sentiment_ = logit(output_sentiment_)
        predictions_sentiment_ = torch.argmax(logits_sentiment_, axis=1)
        predictions_phonem_ = torch.argmax(logits_phonem_, axis=1)
        accuracy_metric.add_batch(predictions=predictions_sentiment_, references = target_sentimen)
        wer_metric.add_batch(predictions=predictions_phonem_, references = target_phonemes)
    print(f"wer metric: {wer_metric.compute()}; accuracy metric: {accuracy_metric.compute()};")





/usr/local/lib/python3.9/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


  0%|          | 0/3615 [00:00<?, ?it/s]

loss1: 10.22; loss2: 11733.20; loss_sum: 11743.41895; iter: 0


<ipython-input-22-ad86ca82496d>:39: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(parameters=mo.parameters(), max_norm=10, norm_type=2.0)


OutOfMemoryError: ignored

In [ ]:
#отдельные строчки по подсчету метрики
from torch import logit
for batch in test_DataLoader:
            batch = {k: v.to(device) for k, v in batch.items()}
            target_phonemes = batch['labels_phonems']
            target_sentimen = batch[ 'labels_sentiment']['input_ids']
            embedding = batch['input_values']
            with torch.no_grad():
              outputs_phonemes_, output_sentiment_  = model(embedding)
              logits_phonem_ = logit(outputs_phonemes_)
              logits_sentiment_ = logit(output_sentiment_)
              predictions_sentiment_ = torch.argmax(logits_sentiment_, axis=1)
              predictions_phonem_ = torch.argmax(logits_phonem_, axis=1)
              accuracy_metric.add_batch(predictions=predictions_sentiment_, references = target_sentimen)
              wer_metric.add_batch(predictions=predictions_phonem_, references = target_phonemes)
print(wer_metric.compute())
accuracy_metric.compute()

0.9433495482974288


{'accuracy': 0.026595744680851064}

In [ ]:
#сохранение модели
model.save_pretrained('/content/drive/MyDrive/multitask')

In [ ]:
#ещё один способ сохранения 
torch.save(model.module.state_dict(), PATH)